In [1]:
import pandas as pd
import xlrd
import re
from datetime import datetime

In [2]:
def stack_dataframe(df, dim_cols, del_cols):
    """
    Função retorna lista de dataframes após processo de unpivot
    :Params:
    df: DataFrame original
    dim_cols: colunas de dimensão
    del_cols: colunas a serem desconsideradas
    """
    dfs = []
    month_mapper = {'Jan': '1',
                    'Fev': '2',
                    'Mar': '3',
                    'Abr': '4',
                    'Mai': '5',
                    'Jun': '6',
                    'Jul': '7',
                    'Ago': '8',
                    'Set': '9',
                    'Out': '10',
                    'Nov': '11',
                    'Dez': '12'}
    for col in [el for el in df.columns if el not in (dim_cols + del_cols)]:
        tmp_df = df[dim_cols[:] + [col]]
        tmp_df['volume'] = tmp_df[col]
        tmp_df['mes'] = tmp_df.apply(lambda row: f'{row["ANO"]}-{month_mapper[col]}', axis=1)
        del tmp_df[col]
        del tmp_df['ANO']
        dfs.append(tmp_df)
    return dfs

In [3]:
def get_unit(value, regex_exp=r'\(([^\)]+)\)', del_chars=['(', ')']):
    """
    Função retorna a unidade de medida do combustível via regex
    :Params:
    value: string do nome do combustível
    regex_exp: expressão regex para captura da medida, padrão é entre parenteses
    del_chars: lista de caracteres a serem retirados do valor de unidade
    """
    res = re.search(regex_exp, value)
    if res:
        unit = res[0]
        for char in del_chars:
            unit = unit.replace(char, '')
        return unit
    return None

## Stacking DataFrames
A célula abaixo parte das planilhas, realizando o processo de unpivot dos dois DataFrames e os concatena ao final

In [4]:
df_fuels = pd.read_excel('vendas-combustiveis-m3.xls', sheet_name=1)
df_diesel = pd.read_excel('vendas-combustiveis-m3.xls', sheet_name=2)

stacked_diesel_df = pd.concat(stack_dataframe(df_diesel, ['COMBUSTÍVEL', 'ANO', 'ESTADO', 'REGIÃO'], ['TOTAL']))
stacked_fuels_df = pd.concat(stack_dataframe(df_fuels, ['COMBUSTÍVEL', 'ANO', 'ESTADO', 'REGIÃO'], ['TOTAL']))

df = pd.concat([stacked_diesel_df, stacked_fuels_df])

<ipython-input-2-8b0112c7b80a>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['volume'] = tmp_df[col]
<ipython-input-2-8b0112c7b80a>:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp_df['mes'] = tmp_df.apply(lambda row: f'{row["ANO"]}-{month_mapper[col]}', axis=1)


## Shaping DataFrame
A célula abaixo formata os dados no padrão requisitados, realizando processos de:
* Renomeação de colunas
* Ajuste de tipo de dados
* Extração de features

In [5]:
df['unit'] = df.apply(lambda row: get_unit(row['COMBUSTÍVEL']), axis=1)
df['year_month'] = df.apply(lambda row: datetime.strptime(row['mes'], '%Y-%m'), axis=1)

agg_df = df.groupby(['year_month', 'ESTADO', 'COMBUSTÍVEL', 'unit']).sum('volume').reset_index()

agg_df = agg_df.rename(columns={'ESTADO': 'uf', 'COMBUSTÍVEL': 'product'})

agg_df['year'] = agg_df['year_month'].dt.year
agg_df['created_at'] = datetime.now()

## Integrity Check
A célula abaixo compara os valores de totais do combustíveis das planilhas originais com o valores no DataFrame resultante do processo de transformação

In [6]:
df_raw = pd.concat([df_diesel, df_fuels])
agg_raw_df = df_raw[['COMBUSTÍVEL', 'TOTAL']].groupby(['COMBUSTÍVEL']).sum('TOTAL')

check_df = agg_df[['product', 'volume']].groupby(['product']).sum('volume')
check_df = check_df.merge(agg_raw_df, left_index=True, right_index=True)
check_df

,volume,TOTAL
product,,
ETANOL HIDRATADO (m3),4.298877e+08,3.406194e+07
GASOLINA C (m3),1.209812e+09,1.022497e+08
GASOLINA DE AVIAÇÃO (m3),2.365254e+06,1.893234e+05
GLP (m3),4.827797e+08,4.201662e+07
QUEROSENE DE AVIAÇÃO (m3),2.210647e+08,1.674294e+07
QUEROSENE ILUMINANTE (m3),1.986087e+06,2.360802e+05
ÓLEO COMBUSTÍVEL (m3),1.942207e+08,1.630406e+07
ÓLEO DIESEL (OUTROS ) (m3),3.927839e+05,2.931939e+04
ÓLEO DIESEL (m3),1.829433e+09,1.551037e+08


## Persisting Data
A célula abaixo persiste o DataFrame em formato Parquet, particionando sobre o produto e ano da informação

In [7]:
agg_df.to_parquet('anp-dataset', partition_cols=['product', 'year'])